In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory

# 1. Load Dataset (replace with your dataset path)
train_ds = image_dataset_from_directory(
    "train",
    image_size=(224, 224),  # ResNet50 expects 224x224 images
    batch_size=32
)

val_ds = image_dataset_from_directory(
    "dataset/val",
    image_size=(224, 224),
    batch_size=32
)

# 2. Normalize pixel values
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# 3. Load Pre-trained ResNet50
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # Freeze all ResNet50 layers

# 4. Add custom classification layers
num_classes = len(train_ds.class_names)

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

# 5. Compile the model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# 6. Train the model
history = model.fit(train_ds, validation_data=val_ds, epochs=5)

# 7. Evaluate the model
loss, acc = model.evaluate(val_ds)
print(f"✅ Validation Accuracy: {acc:.2f}")


Found 0 files belonging to 0 classes.


ValueError: No images found in directory train. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [3]:
!pip install tensorflow